---
tags:
    - UsdStage
    - Usd
    - Python
---
Stageの使い方いろいろテスト

In [1]:
from pxr import Usd,Sdf

In [37]:
# 既存のUSDファイルをロードする
stage = Usd.Stage.Open("usd/sample.usda")
# 一度ロードしたものをそのあと編集すると、編集した内容がキャッシュされているので
# クリアしたい場合はReloadする
stage.Reload()
# RootLayerをPrintできる
print(stage.ExportToString())

#usda 1.0
(
    defaultPrim = "Sample"
    doc = """Generated from Composed Stage of root layer c:\\reincarnation_tech\\notebooks\\USD\\Stage\\usd\\sample.usda
"""
)

def "Sample"
{
}




In [38]:
# DefaultPrimを取得して、Attributeを追加する
print(stage.HasDefaultPrim()) # DefaultPrimを持っているか
prim = stage.GetDefaultPrim()
prim.CreateAttribute("hoge",Sdf.ValueTypeNames.String).Set("Fuga")
layer = stage.GetRootLayer()

True


In [39]:
# TimeCode指定
stage.SetStartTimeCode(1)
stage.SetEndTimeCode(30)
print(stage.ExportToString())

#usda 1.0
(
    defaultPrim = "Sample"
    doc = """Generated from Composed Stage of root layer c:\\reincarnation_tech\\notebooks\\USD\\Stage\\usd\\sample.usda
"""
    endTimeCode = 30
    startTimeCode = 1
)

def "Sample"
{
    custom string hoge = "Fuga"
}




In [40]:
# セッションレイヤーを取得して、EditTargetにする
session = stage.GetSessionLayer()
target = Usd.EditTarget(session)
stage.SetEditTarget(target)
session.subLayerPaths = [layer.identifier]

In [41]:
# Targetは、ステージのEdit対象レイヤーのこと。
print(target.GetLayer().identifier)
# どこのSdfPathにMapされるか
print(target.GetMapFunction())
# TargetのレイヤーからPrimSpecを取得する
print(target.GetPrimSpecForScenePath("/Sample"))

anon:000001B2F688ADF0:sample-session.usda
/ -> /
None


In [42]:
# Prim定義
prim = stage.DefinePrim("/SampleB")
# Overを定義
over = stage.OverridePrim("/Sample")
over.CreateAttribute("fuga",Sdf.ValueTypeNames.String).Set("hoge")

True

In [43]:
print(session.ExportToString())

#usda 1.0
(
    subLayers = [
        @usd/sample.usda@
    ]
)

def "SampleB"
{
}

over "Sample"
{
    custom string fuga = "hoge"
}




In [44]:
print(layer.ExportToString())

#usda 1.0
(
    defaultPrim = "Sample"
    endTimeCode = 30
    startTimeCode = 1
)

def "Sample"
{
    custom string hoge = "Fuga"
}




In [45]:
print(stage.ExportToString())

#usda 1.0
(
    defaultPrim = "Sample"
    doc = """Generated from Composed Stage of root layer c:\\reincarnation_tech\\notebooks\\USD\\Stage\\usd\\sample.usda
"""
    endTimeCode = 30
    startTimeCode = 1
)

def "Sample"
{
    custom string fuga = "hoge"
    custom string hoge = "Fuga"
}

def "SampleB"
{
}




In [46]:
# SessionLayerをミュートする
stage.MuteLayer(session.identifier)
print(stage.ExportToString())

#usda 1.0
(
    defaultPrim = "Sample"
    doc = """Generated from Composed Stage of root layer c:\\reincarnation_tech\\notebooks\\USD\\Stage\\usd\\sample.usda
"""
    endTimeCode = 30
    startTimeCode = 1
)

def "Sample"
{
    custom string hoge = "Fuga"
}




In [47]:
# UnmuteしてTraverse
stage.UnmuteLayer(session.identifier)
for prim in stage.Traverse():
    print(prim)

Usd.Prim(</Sample>)
Usd.Prim(</SampleB>)


In [48]:
# Memory上にレイヤーを作成する
memStage = Usd.Stage.CreateInMemory()
print(memStage.ExportToString())

#usda 1.0
(
    doc = """Generated from Composed Stage of root layer 
"""
)




In [49]:
# レイヤースタックを取得
for stack in stage.GetLayerStack():
    print(stack)

Sdf.Find('anon:000001B2F688ADF0:sample-session.usda')
Sdf.Find('usd/sample.usda')
Sdf.Find('usd/sample.usda')


In [36]:
# レイヤーを取得
print(stage.GetUsedLayers())

[Sdf.Find('anon:000001B2F688D8F0:sample-session.usda'), Sdf.Find('usd/sample.usda')]
